# Проект "SQL"

**Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.**

1. **Таблица books Содержит данные о книгах:**
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.
2. **Таблица authors Содержит данные об авторах:**
- author_id — идентификатор автора;
- author — имя автора.
3. **Таблица publishers Содержит данные об издательствах:**
- publisher_id — идентификатор издательства;
- publisher — название издательства;
4. **Таблица ratings Содержит данные о пользовательских оценках книг:**
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.
5. **Таблица reviews Содержит данные о пользовательских обзорах на книги:**
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

In [1]:
!pip install psycopg2-binary

In [2]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

In [4]:
# формируем запрос и выводим данные
query = '''SELECT *, COUNT(*) OVER() AS table_size FROM books LIMIT 5'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id,table_size
0,1,546,'Salem's Lot,594,2005-11-01,93,1000
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336,1000
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135,1000
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309,1000
4,5,125,1776,386,2006-07-04,268,1000


In [5]:
query = '''SELECT count(*) FROM books'''
get_sql_data(query)

,count
0,1000


In [6]:
query = '''SELECT * FROM authors LIMIT 5'''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
query = '''SELECT count(*) FROM authors '''
get_sql_data(query)

,count
0,636


In [8]:
query = '''SELECT * FROM publishers LIMIT 5'''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
query = '''SELECT count(*) FROM publishers '''
get_sql_data(query)

,count
0,340


In [10]:
query = '''SELECT * FROM ratings LIMIT 5'''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
query = '''SELECT count(*) FROM ratings'''
get_sql_data(query)

,count
0,6456


In [12]:
query = '''SELECT * FROM reviews LIMIT 5'''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [13]:
query = '''SELECT count(*) FROM reviews '''
get_sql_data(query)

,count
0,2793


**Загрузил и проверил таблицы. Таблица books Содержит 1000 строк, Таблица autors Содержит 639, Таблица publisher Содержит 340, Таблица rating Содержит 6454, Таблица reviews Содержит 2793, строки**

## Задания
1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайтетолько книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставилибольше 48 оценок.


*Посчитайте, сколько книг вышло после 1 января 2000 года*

In [14]:
query = '''WITH t as (SELECT * 
            FROM books
            where EXTRACT(YEAR FROM CAST(publication_date AS date))> 2000-01-01)
            SELECT count(distinct(book_id))
            FROM t'''
get_sql_data(query)

,count
0,862


**C 1 января 2000 было выпущено 862 книга**

In [15]:
# ячейка ревьюера:
query = '''WITH t as (SELECT * 
            FROM books
            WHERE publication_date > '2000-01-01')
            SELECT count(distinct(book_id))
            FROM t'''
get_sql_data(query)

,count
0,819


*Для каждой книги посчитайте количество обзоров и среднюю оценку;*

In [16]:
query = '''SELECT b.book_id, title, COUNT(DISTINCT(review_id)) AS count_review,  ROUND(AVG(rating),2) AS rating_avg
            FROM books as b
            JOIN reviews as r on b.book_id=r.book_id
            JOIN ratings as ra on b.book_id=ra.book_id
            GROUP BY b.book_id
            LIMIT 5
            '''
get_sql_data(query)

,book_id,title,count_review,rating_avg
0,1,'Salem's Lot,2,3.67
1,2,1 000 Places to See Before You Die,1,2.50
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,4,1491: New Revelations of the Americas Before C...,2,4.50
4,5,1776,4,4.00


**Таблица содержит id книг, название,  количество обзоров и среднюю оценку**

*Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры*

In [17]:
query = '''SELECT publisher, COUNT(book_id)
            FROM books as b        
            JOIN publishers as p on b.publisher_id=p.publisher_id
            WHERE num_pages>50
            GROUP BY publisher
            ORDER BY count DESC
            LIMIT 1
            '''
get_sql_data(query)

,publisher,count
0,Penguin Books,42


**Наибольшее количество книг, толщиной более 50 листов выпустило издательство "Penguin Books" 42 книги**

*Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками*

In [18]:
query = '''WITH t AS (SELECT *
            FROM books as b        
            JOIN authors as a on b.author_id=a.author_id
            JOIN ratings as r on b.book_id=r.book_id
            WHERE b.book_id IN (SELECT book_id
        FROM ratings
        GROUP BY book_id
        HAVING COUNT(rating) >= 50))
        SELECT author, ROUND(AVG(rating),2) AS rating_avg
        FROM t
        GROUP BY t.author
        ORDER BY rating_avg desc
        LIMIT 1
            '''
get_sql_data(query)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.29


**Получили автора с самой высокой средней оценкой, сучетом того что, у книги не менее 50 оценок**

In [19]:
# ячейка ревьюера:
query = '''

WITH t AS (
    SELECT 
        a.author,
        b.book_id,
        AVG(r.rating) AS book_avg_rating
    FROM books as b        
    JOIN authors as a on b.author_id=a.author_id
    JOIN ratings as r on b.book_id=r.book_id
    WHERE b.book_id IN (
        SELECT book_id
        FROM ratings
        GROUP BY book_id
        HAVING COUNT(rating) >= 50)
    GROUP BY 
        a.author,
        b.book_id
)
SELECT 
    author, 
    AVG(book_avg_rating) AS rating_avg
FROM t
GROUP BY t.author
ORDER BY rating_avg desc
LIMIT 1

'''
get_sql_data(query)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.283844


*Посчитайте среднее количество обзоров от пользователей, которые поставилибольше 48 оценок.*

In [20]:
query = ''' WITH t as (SELECT username, COUNT(text)
            FROM reviews 
            WHERE username IN (SELECT username
                FROM ratings 
                GROUP BY username
                HAVING COUNT(rating) > 48)
            GROUP BY username)
            SELECT AVG(count)
            FROM t
            '''
get_sql_data(query)

,avg
0,24.0


**Посчитал среднее количество обзоров от пользователей, которые поставили больше 48 оценок.**

**Общий вывод:**
1. Были загружены и проверены данные
2. Решены технические задачи. В ходе выполнения получены следующие результаты:
   - C 1 января 2000 было выпущено 862 книга
   - Выведена таблица которая содержит id книг, название,  количество обзоров и среднюю оценку
   - Наибольшее количество книг, толщиной более 50 листов выпустило издательство "Penguin Books" 42 книги
   - Получили автора с самой высокой средней оценкой, сучетом того что, у книги не менее 50 оценок
   - Посчитал среднее количество обзоров от пользователей, которые поставили больше 48 оценок, их получилось 24